In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import mpld3
mpld3.enable_notebook()

In [3]:
def read_json(name):
    print(name)
    with open(name) as data_file:    
            data = json.load(data_file)
    return data

In [4]:
def data_proces(name,Deface=100,NumSam=200):
    datos=read_json(name)
    entradas=read_json('_'.join(('inp',name)))
    train=np.array([[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]])
    referencia=[0]
    for key in datos.keys():
        print(len(datos[key]))
        for k,exp in enumerate(datos[key]):
            VelC=exp["velocity"]
            forceC=exp["force"]
            VelCj=[round(elem, 1) for elem in VelC]
            Conj=set(VelCj)
            infbar=100
            for elem in list(Conj):
                indices=[i for i,x in enumerate(VelCj) if x==elem]
                for ind in range(len(indices)-1):
                    if forceC[indices[ind]] == forceC[indices[ind+1]] and indices[ind]> 120:
                        reference = [indices[ind],indices[ind+1]]
                        break
                    elif abs(forceC[indices[ind]]-forceC[indices[ind+1]])<infbar and indices[ind+1]-indices[ind]>20 and indices[ind+1]-indices[ind]<200 and indices[ind]> 120:
                        tempref = [indices[ind],indices[ind+1]]
                        infbar = abs(forceC[indices[ind]]-forceC[indices[ind+1]])
                if 'reference' in locals():
                    break
            
            if 'reference' in locals():    
                Deface = reference[0]
                NumSam = reference[1] - reference[0]
                del reference
            else:
                Deface = tempref[0]
                NumSam = tempref[1] - tempref[0]
                del tempref
            referencia.append(referencia[-1]+NumSam+1)
            for element in range(NumSam+1):
                caso=list()
                caso.append(entradas[key]["Ap"])
                caso.append(entradas[key]["Ag"])
                caso.append(entradas[key]["h"])
                caso.append(entradas[key]["L"])
                caso.append(entradas[key]["viscocidad"])
                caso.append(entradas[key]["Ty"][k]/1000)
                caso.append((exp["velocity"][element+Deface])*0.001)
                caso.append(exp["displace"][element+Deface])
                caso.append(exp["force"][element+Deface])
                casonp=np.array(caso)
                #print(len(casonp))
                train=np.append(train,[casonp],axis=0)
            
    return train[2:][:],referencia

In [5]:
def Target_C(data):
    Arn=(data[0]+((data[1])/2))
    Fn=((12*(data[4])*Arn*(data[6]/10))/((data[1])*data[2]*data[2]))
    Fn=Fn*(data[0]*data[3])
    Fty=data[8]-Fn
    c=((Fty*data[2])/(data[3]*data[0]*data[5]*1000))
    return Fty

In [6]:
def force_limit(data):
    Arn=(data[0]+((data[1])/2))
    Fn=((12*(data[4])*Arn*(data[6]))/((data[1])*data[2]*data[2]))
    Fn=Fn*(data[0]*data[3])
    #c=2.07+((12*data[4]*data[0]*data[6])/((12*data[4]*data[0]*data[6])+(0.4*data[1]*data[2]*data[5])))
    c=1.07
    Fty= (c*data[3]*data[0]*data[5]*1000)/(data[2])
    fr=Fn+Fty
    #print(fr)
    #print(data[8])
    return fr

In [7]:
train,refer=data_proces("twofluidexp.json",Deface=120,NumSam=260)
#print(train.shape)
#print(train[1:4])
np.random.shuffle(train)
#train[1:4,8:9].shape
#print(train[20][])
print(refer)

twofluidexp.json
inp_twofluidexp.json
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
[0, 121, 242, 368, 498, 623, 745, 869, 993, 1115, 1239, 1361, 1485, 1605, 1731, 1854, 1973, 2098, 2223, 2348, 2465, 2589, 2712, 2833, 2959, 3082, 3207, 3328, 3447, 3570, 3694, 3816, 3938, 4060, 4185, 4311, 4436, 4562, 4687, 4812, 4935, 5053, 5179, 5303, 5424, 5550, 5673, 5796, 5920, 6045, 6170, 6293, 6415, 6538, 6661, 6784, 6905, 7028, 7152, 7275, 7398, 7524, 7651, 7774, 7896, 8015, 8135, 8260, 8386, 8510, 8632, 8755, 8879, 9004, 9124, 9241, 9366, 9491, 9615, 9738, 9864, 9988, 10113, 10239, 10364, 10485, 10610, 10735, 10855, 10973, 11097, 11223, 11349, 11474, 11598, 11721, 11841, 11965, 12090, 12215, 12340, 12463, 12588, 12717, 12843, 12957, 13083, 13208, 13333]


In [8]:
# Agregar funcion para limpiar la gráfica 
# Evita duplicar elementos 
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [9]:
def Deep_neural_network(
    learning_rate = 1e-2,
    batch_size=20,
    Drop_prob=1.0,
    n_nodes_hl1 = 30,
    n_nodes_hl2 = 30,
    n_nodes_hl3 = 30,
    n_nodes_hl4 = 30,
    beta=.01,
    n_classes=1):
    
    # limpiar gráficas anteriores 
    reset_graph()
    
    #input place holder 
    x= tf.placeholder(tf.float32,[batch_size,8], name='input_placeholder')
    #output place holder
    y =tf.placeholder(tf.float32,[batch_size,1],name='output_placeholder')
    
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([8, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    hidden_4_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_nodes_hl4])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl4]))}
    


    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}
    
    l1 = tf.add(tf.matmul(x,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    l1 = tf.nn.dropout(l1,Drop_prob)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    l2 = tf.nn.dropout(l2,Drop_prob)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    l3 = tf.nn.dropout(l3,Drop_prob)
    
    l4 = tf.add(tf.matmul(l3,hidden_4_layer['weights']), hidden_4_layer['biases'])
    l4 = tf.nn.relu(l4)
    l4 = tf.nn.dropout(l4,Drop_prob)
    
    #l5 = tf.add(tf.matmul(l4,hidden_5_layer['weights']), hidden_5_layer['biases'])
    #l5 = tf.nn.relu(l5)
    #l5 = tf.nn.dropout(l5,Drop_prob)

    output = tf.matmul(l2,output_layer['weights']) + output_layer['biases']
    
    regularizer = tf.nn.l2_loss(hidden_1_layer['weights']) + tf.nn.l2_loss(hidden_2_layer['weights'])+tf.nn.l2_loss(hidden_3_layer['weights']) + tf.nn.l2_loss(hidden_4_layer['weights'])
    
    total_loss = tf.losses.mean_squared_error(output, y)
   
    tf.summary.scalar("Lost",total_loss)
    
    loss = tf.reduce_mean(total_loss + beta * regularizer)
    
    tf.summary.scalar("Regularized_loss",loss)
    
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)
    
    #generar logs
    summaries = tf.summary.merge_all()
    
    
    return dict(
        x = x,
        y = y,
        output=output,
        total_loss = total_loss,
        train_step = train_step,
        summaries=summaries,
        saver = tf.train.Saver() 
    )
    
            

In [10]:
Deep_neural_network()

{'output': <tf.Tensor 'add:0' shape=(20, 1) dtype=float32>,
 'saver': <tensorflow.python.training.saver.Saver at 0x7fb09d63ef28>,
 'summaries': <tf.Tensor 'Merge/MergeSummary:0' shape=() dtype=string>,
 'total_loss': <tf.Tensor 'mean_squared_error/value:0' shape=() dtype=float32>,
 'train_step': <tf.Operation 'Adam' type=NoOp>,
 'x': <tf.Tensor 'input_placeholder:0' shape=(20, 8) dtype=float32>,
 'y': <tf.Tensor 'output_placeholder:0' shape=(20, 1) dtype=float32>}

In [11]:
#def conv2d(x, W):
#    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

#def maxpool2d(x):
    #                        size of window         movement of window
#    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [12]:
#convolutional_neural_network()

In [13]:
def train_network(g,data, num_epochs, num_steps = 15, batch_size = 20, verbose = True, save=False
                  ,batch_pick=22,checkpoint=False,step=0):
    tf.set_random_seed(2345)
    print('Start training')
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        writer = tf.summary.FileWriter("./LOGTEST")
        tf.summary.FileWriter.add_graph(writer,sess.graph)
        
        if checkpoint:
            g['saver'].restore(sess, checkpoint)
        training_losses = []
        for epoch in range(num_epochs):
            epoch_loss = 0
            dim,var=data.shape
            for j in range(int(dim/batch_size)):

                epoch_x=data[batch_size*(j):batch_size*(j+1),0:8]
                
                epoch_Y=data[batch_size*(j):batch_size*(j+1),8:9]
                
                    
                feed_dict={g['x']: epoch_x, 
                           g['y']: epoch_Y
                          }
                
                output,training_loss_, training_state,summ = sess.run([g['output'],g['total_loss'],
                                                      g['train_step'],g["summaries"]],
                                                             feed_dict)
                epoch_loss += training_loss_
                
            if (epoch%2 == 0):
                writer.add_summary(summ,step+epoch)
                
                    
            if verbose:
                print("Average training loss for Epoch", epoch, ":", epoch_loss)
                #print(output)
            training_losses.append(epoch_loss)

        if isinstance(save, str):
            g['saver'].save(sess, save)

    return training_losses,step+epoch

In [14]:
Paso=0


In [15]:

tf.reset_default_graph()
g = Deep_neural_network(learning_rate = 1e-2,batch_size=100,Drop_prob=1)
t = time.time()
print(train.shape)
losses,Paso = train_network(g, train, 500,batch_size=100,verbose =False ,save="./One_2*30_2fluid.ckpt",step=Paso)
#losses,Paso = train_network(g, train, 1000,batch_size=100,verbose =False,save="./Exp_1*50_2fluid.ckpt",checkpoint="./Exp_1*50_2fluid.ckpt",step=Paso)
#losses = train_network(g, train, 1000,batch_size=1,verbose =False,save="./Twocase1.ckpt",checkpoint="./OneCase.ckpt")
print("It took", time.time() - t, "seconds to train.")
print("The average loss on the final epoch was:", losses[-1])

(13333, 9)
Start training


InternalError: Blas GEMM launch failed : a.shape=(100, 8), b.shape=(8, 30), m=100, n=30, k=8
	 [[Node: MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](_arg_input_placeholder_0_0/_1, Variable/read)]]

Caused by op 'MatMul', defined at:
  File "/home/sia/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sia/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/sia/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/sia/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/sia/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py"